<a href="https://colab.research.google.com/github/CrysG011/Criptografia/blob/main/generador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import sqlite3
import random
import string
import os
import binascii

# Función para generar contraseñas aleatorias
def generar_password():
    longitud = 23
    caracteres = string.ascii_letters + string.digits + string.punctuation
    password = ''.join(random.choice(caracteres) for i in range(longitud))
    return password

# Función para generar hash de contraseñas
def generar_hash(password):
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    hash_password = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), salt, 100000)
    hash_password = binascii.hexlify(hash_password)
    return (salt + hash_password).decode('ascii')

# Conexión a la base de datos
conn = sqlite3.connect('passwords.db')
c = conn.cursor()

# Creación de la tabla para almacenar contraseñas
c.execute('''CREATE TABLE IF NOT EXISTS passwords
             (id INTEGER PRIMARY KEY, password TEXT, hash TEXT)''')

# Función para agregar una contraseña a la base de datos
def agregar_password(password):
    hash_password = generar_hash(password)
    c.execute("INSERT INTO passwords (password, hash) VALUES (?, ?)", (password, hash_password))
    conn.commit()

# Función para verificar una contraseña ingresada
def verificar_password(password):
    c.execute("SELECT hash FROM passwords WHERE password = ?", (password,))
    row = c.fetchone()
    if row:
        stored_hash = row[0]
        salt = stored_hash[:64]
        stored_hash = stored_hash[64:]
        hash_password = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), salt.encode('ascii'), 100000)
        hash_password = binascii.hexlify(hash_password).decode('ascii')
        if hash_password == stored_hash:
            return True
    return False

# Ejemplo de uso del generador de contraseñas seguras
password = generar_password()
print(f"Contraseña generada: {password}")
agregar_password(password)

# Verificación de una contraseña ingresada
password_ingresada = input("Ingrese la contraseña para verificar: ")
if verificar_password(password_ingresada):
    print("La contraseña es correcta.")
else:
    print("La contraseña es incorrecta.")



Contraseña generada: uA0_a5GVg)YO
Ingrese una contraseña para verificar: uA0_a5GVg)YO
La contraseña es correcta.
